# Query SRA metadata using Big Query

Using https://pandas-gbq.readthedocs.io/. 

Before you begin, you must create a Google Cloud Platform project. Use the BigQuery sandbox to try the service for free.

If you do not provide any credentials, this module attempts to load credentials from the environment. If no credentials are found, pandas-gbq prompts you to open a web browser, where you can grant it permissions to access your cloud resources. These credentials are only used locally.

In [1]:
import pandas_gbq
import pandas as pd

In [2]:
# Construct query
# Here we are using the bioproject PRJNA523380 for the CCLE cell lines
# I only want to get the RNA-seq data 
query = """
SELECT * FROM `nih-sra-datastore.sra.metadata` 
WHERE bioproject = 'PRJNA523380' AND assay_type = 'RNA-Seq'
LIMIT 5000
"""

In [3]:
# Query the metadata table in BigQuery
# Might take some time
# Returns a dataframe
df = pd.read_gbq(query, dialect="standard")

In [6]:
df

,acc,assay_type,center_name,consent,experiment,sample_name,instrument,librarylayout,libraryselection,librarysource,...,geo_loc_name_sam,ena_first_public_run,ena_last_update_run,sample_name_sam,datastore_filetype,datastore_provider,datastore_region,attributes,run_file_version,jattr
0,SRR8615408,RNA-Seq,BROAD INSTITUTE,public,SRX5414345,AMO1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[bam, run.zq, sra]","[gs, s3, ncbi]","[s3.us-east-1, gs.US, ncbi.public]","[{'k': 'sex_calc', 'v': 'female'}, {'k': 'asse...",1,"{""sex_calc"": ""female"", ""assemblyname"": ""GCA_00..."
1,SRR8615898,RNA-Seq,BROAD INSTITUTE,public,SRX5415152,NCIH1299_LUNG,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[bam, sra, run.zq]","[gs, s3, ncbi]","[gs.US, ncbi.public, s3.us-east-1]","[{'k': 'sex_calc', 'v': 'male'}, {'k': 'assemb...",1,"{""sex_calc"": ""male"", ""assemblyname"": ""GCA_0000..."
2,SRR8616188,RNA-Seq,BROAD INSTITUTE,public,SRX5414862,NCIH146_LUNG,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[sra, run.zq, bam]","[gs, s3, ncbi]","[ncbi.public, gs.US, s3.us-east-1]","[{'k': 'sex_calc', 'v': 'male'}, {'k': 'assemb...",1,"{""sex_calc"": ""male"", ""assemblyname"": ""GCA_0000..."
3,SRR8615664,RNA-Seq,BROAD INSTITUTE,public,SRX5414736,HUH1_LIVER,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[sra, bam, run.zq]","[gs, s3, ncbi]","[gs.US, ncbi.public, s3.us-east-1]","[{'k': 'sex_calc', 'v': 'male'}, {'k': 'assemb...",1,"{""sex_calc"": ""male"", ""assemblyname"": ""GCA_0000..."
4,SRR8615310,RNA-Seq,BROAD INSTITUTE,public,SRX5414443,SNU398_LIVER,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[run.zq, bam, sra]","[gs, s3, ncbi]","[gs.US, s3.us-east-1, ncbi.public]","[{'k': 'sex_calc', 'v': 'male'}, {'k': 'assemb...",1,"{""sex_calc"": ""male"", ""assemblyname"": ""GCA_0000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014,SRR8615714,RNA-Seq,BROAD INSTITUTE,public,SRX5414686,KALS1_CENTRAL_NERVOUS_SYSTEM,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[run.zq, sra, bam]","[ncbi, s3, gs]","[s3.us-east-1, gs.US, ncbi.public]","[{'k': 'sex_calc', 'v': 'female'}, {'k': 'asse...",1,"{""sex_calc"": ""female"", ""assemblyname"": ""GCA_00..."
1015,SRR8615590,RNA-Seq,BROAD INSTITUTE,public,SRX5414810,RERFLCKJ_LUNG,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[run.zq, sra, bam]","[ncbi, s3, gs]","[gs.US, s3.us-east-1, ncbi.public]","[{'k': 'sex_calc', 'v': 'male'}, {'k': 'assemb...",1,"{""sex_calc"": ""male"", ""assemblyname"": ""GCA_0000..."
1016,SRR8615630,RNA-Seq,BROAD INSTITUTE,public,SRX5414770,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[bam, run.zq, sra]","[ncbi, s3, gs]","[ncbi.public, gs.US, s3.us-east-1]","[{'k': 'sex_calc', 'v': 'male'}, {'k': 'assemb...",1,"{""sex_calc"": ""male"", ""assemblyname"": ""GCA_0000..."
1017,SRR8615730,RNA-Seq,BROAD INSTITUTE,public,SRX5414670,TTC549_SOFT_TISSUE,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[sra, bam, run.zq]","[ncbi, s3, gs]","[gs.US, s3.us-east-1, ncbi.public]","[{'k': 'sex_calc', 'v': 'NA'}, {'k': 'assembly...",1,"{""sex_calc"": ""NA"", ""assemblyname"": ""GCA_000001..."


# Goal here is to curate metadata for the results of the query
I specifically want a dataframe with the desired metadata as well as a dictionary with the same information.

Hoping to be able to use the dictionary to add the metadata to the objects in the the GCS bucket 

In [4]:
#  TODO:: refactor these functions to be more generic
# Also improve efficiency by not iterating over the entire dataframe
# Maybe use a dictionary comprehension instead

def convert_array_to_dict(arr):
    result_dict = {}
    desired_keys = ['bases', 'bytes', 'run_file_create_date', 'disease_sam', 'disease_stage_sam_s_dpl172', 'tissue_sam']
    for item in arr:
        if isinstance(item, dict):
            key = item.get('k')
            value = item.get('v')
            if key in desired_keys:
                if key == 'bytes':
                    result_dict['size_in_bytes'] = int(value)
                    result_dict['size_in_GB'] = round(float(value) / 1000000000, 2)
                elif key == 'run_file_create_date':
                    result_dict[key] = str(value)
                else:
                    if key in result_dict:
                        if isinstance(result_dict[key], list):
                            result_dict[key].append(value)
                        else:
                            result_dict[key] = [result_dict[key], value]
                    else:
                        result_dict[key] = value
    return result_dict

def convert_row_to_dict(row):
    result_dict = {}
    for column in row.index:
        if column == 'attributes':
            result_dict.update(convert_array_to_dict(row[column]))
        elif column == 'releasedate':
            # convert type Timestamp to string
            result_dict[column] = str(row[column])
        elif column == 'run_file_create_date':
            result_dict[column] = str(row[column])
        else:
            result_dict[column] = row[column]
    return result_dict


def convert_dataframe_to_dict(df):
    result_dict = {}
    for i in range(len(df)):
        result_dict[df['acc'][i]] = convert_row_to_dict(df.iloc[i])
    return result_dict


In [5]:
# choose columns
columns = ['acc', 'sample_name', 'sample_acc', 'experiment',  'library_name', 'sra_study', 'center_name', 
'platform', 'assay_type', 'librarysource', 'organism', 'releasedate']

# subset the dataframe to only include the columns we want
df_ = df[columns + ['attributes']].copy()

# # for each column in columns, print out the number of unique values, and then the first 5 unique values
for col in columns:
    # print(f"For {col}, there are: {df[col].nunique()} unique values. \nExamples: {df[col].unique()[0:5]}\n")
    print(f"{col} has {df[col].nunique()} unique values. \nExamples: {df[col].unique()[0:5]}\n")

acc has 1019 unique values. 
Examples: ['SRR8615408' 'SRR8615898' 'SRR8616188' 'SRR8615664' 'SRR8615310']

sample_name has 1019 unique values. 
Examples: ['AMO1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE' 'NCIH1299_LUNG' 'NCIH146_LUNG'
 'HUH1_LIVER' 'SNU398_LIVER']

sample_acc has 1019 unique values. 
Examples: ['SRS4395396' 'SRS4396049' 'SRS4395814' 'SRS4395725' 'SRS4395475']

experiment has 1019 unique values. 
Examples: ['SRX5414345' 'SRX5415152' 'SRX5414862' 'SRX5414736' 'SRX5414443']

library_name has 1019 unique values. 
Examples: ['RNASeq-AMO1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE' 'RNASeq-NCIH1299_LUNG'
 'RNASeq-NCIH146_LUNG' 'RNASeq-HUH1_LIVER' 'RNASeq-SNU398_LIVER']

sra_study has 1 unique values. 
Examples: ['SRP186687']

center_name has 1 unique values. 
Examples: ['BROAD INSTITUTE']

platform has 1 unique values. 
Examples: ['ILLUMINA']

assay_type has 1 unique values. 
Examples: ['RNA-Seq']

librarysource has 1 unique values. 
Examples: ['TRANSCRIPTOMIC']

organism has 1 unique valu

In [ ]:
# Subset df to df_ using columns but also include the 'attributes' column
dict_metadata = convert_dataframe_to_dict(df_)
dict_metadata

In [ ]:
# convert dict to pandas dataframe
df_metadata = pd.DataFrame.from_dict(dict_metadata, orient='index')
# rename the 'acc' column to 'run_accession'
df_metadata.rename(columns={'acc': 'run_accession'}, inplace=True)
df_metadata

In [ ]:
# # save the dataframe to a csv file 
# df_metadata.to_csv('../../metadata/sra_metadata.csv', index=False)

# # save the dictionary to a json file
# import json
# with open('../../metadata/sra_metadata.json', 'w') as fp:
#     json.dump(dict_metadata, fp)